In [29]:
# тут если надо устанавливаем xgboost и catboost
!pip install --user --upgrade catboost

  Using cached https://files.pythonhosted.org/packages/20/de/d51d702abb403a8e250339205a7258c37c584da800a22b381b1539ad8691/catboost-0.11.1-cp36-none-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


In [ ]:
!pip install xgboost

Устанавливаем функцию для вывода load bar

In [ ]:
!pip install tqdm

Импотрируем библиотеки для работы с данными

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab

In [2]:
bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')

In [3]:
bioresponce.head()

Activity        D1        D2    D3   D4        D5        D6        D7  \
0         1  0.000000  0.497009  0.10  0.0  0.132956  0.678031  0.273166   
1         1  0.366667  0.606291  0.05  0.0  0.111209  0.803455  0.106105   
2         1  0.033300  0.480124  0.00  0.0  0.209791  0.610350  0.356453   
3         1  0.000000  0.538825  0.00  0.5  0.196344  0.724230  0.235606   
4         0  0.100000  0.517794  0.00  0.0  0.494734  0.781422  0.154361   

         D8        D9  ...    D1767  D1768  D1769  D1770  D1771  D1772  D1773  \
0  0.585445  0.743663  ...        0      0      0      0      0      0      0   
1  0.411754  0.836582  ...        1      1      1      1      0      1      0   
2  0.517720  0.679051  ...        0      0      0      0      0      0      0   
3  0.288764  0.805110  ...        0      0      0      0      0      0      0   
4  0.303809  0.812646  ...        0      0      0      0      0      0      0   

   D1774  D1775  D1776  
0      0      0      0  
1      0      1      0  
2      0      0      0  
3      0      0      0  
4      0      0      0  

[5 rows x 1777 columns]

In [4]:
bioresponce_target = bioresponce.Activity.values

In [5]:
bioresponce_data = bioresponce.iloc[:, 1:]

# Модель RandomForest
### Посмотрим на зависимость качества от количества деревьев

In [6]:
n_trees = [i for i in range(10, 55, 5)]

Импортируем алгоритм для кросс валидации и RandomForest

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [8]:
%%time
scoring = []
for n_tree in n_trees:
    estimator = RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
    score = cross_val_score(estimator, bioresponce_data, bioresponce_target, 
                                             scoring = 'accuracy', cv = 3)    
    scoring.append(score)
scoring = np.asmatrix(scoring)

CPU times: user 11.3 s, sys: 490 ms, total: 11.8 s
Wall time: 12.1 s


In [9]:
scoring

matrix([[0.75859313, 0.7592    , 0.7504    ],
        [0.78097522, 0.7616    , 0.7592    ],
        [0.78417266, 0.7736    , 0.7648    ],
        [0.78257394, 0.7736    , 0.7704    ],
        [0.78816946, 0.7752    , 0.7736    ],
        [0.78816946, 0.7856    , 0.7776    ],
        [0.78896882, 0.78      , 0.7832    ],
        [0.78976819, 0.7856    , 0.7896    ],
        [0.79536371, 0.7816    , 0.7928    ]])

In [10]:
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

# Модель Xgboost

Импортируем функцию для вывода load bar

In [11]:
from tqdm import tqdm_notebook

Импортируем xgboost

In [12]:
import xgboost as xgb

In [13]:
%%time
xgb_scoring = []
for n_tree in tqdm_notebook(n_trees):
    estimator = xgb.XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=n_tree, min_child_weight=3)
    score = cross_val_score(estimator, bioresponce_data, bioresponce_target, 
                                             scoring = 'accuracy', cv = 3)    
    xgb_scoring.append(score)
xgb_scoring = np.asmatrix(xgb_scoring)

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))


CPU times: user 3min 41s, sys: 1.98 s, total: 3min 42s
Wall time: 3min 44s


In [14]:
xgb_scoring

matrix([[0.77617906, 0.7752    , 0.7688    ],
        [0.77857714, 0.7744    , 0.7768    ],
        [0.7873701 , 0.7784    , 0.7768    ],
        [0.79216627, 0.7736    , 0.7832    ],
        [0.79776179, 0.7776    , 0.7824    ],
        [0.79616307, 0.7816    , 0.78      ],
        [0.79296563, 0.7848    , 0.7792    ],
        [0.79856115, 0.7832    , 0.7808    ],
        [0.79936051, 0.7832    , 0.7832    ]])

In [15]:
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.plot(n_trees, xgb_scoring.mean(axis = 1), marker='.', label='XGBoost')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

# Catboost

In [23]:
from catboost import CatBoostClassifier
from catboost import datasets

In [36]:
titanic_data = pd.read_csv('titanic.csv')

In [37]:
titanic_data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [38]:
# удаляем идентификаторы пассажиров, имена, номер билета и кабины
del titanic_data['PassengerId']
del titanic_data['Name']
del titanic_data['Ticket']
del titanic_data['Cabin']

In [39]:
titanic_data.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [66]:
titanic_data.dropna(inplace=True)

In [67]:
# выделяем колонку ответов
y = titanic_data.Survived
X = titanic_data.drop('Survived', axis=1)

In [68]:
# изменяем тип с int на float
X.Pclass = X.Pclass.astype('float')
X.SibSp = X.SibSp.astype('float')
X.Parch = X.Parch.astype('float')

In [69]:
X.head()

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0     3.0    male  22.0    1.0    0.0   7.2500        S
1     1.0  female  38.0    1.0    0.0  71.2833        C
2     3.0  female  26.0    0.0    0.0   7.9250        S
3     1.0  female  35.0    1.0    0.0  53.1000        S
4     3.0    male  35.0    0.0    0.0   8.0500        S

In [70]:
X.dropna(inplace=True)

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.15)

In [79]:
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.05,
    depth=8,
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1']
)

In [80]:
model.fit(
    Xtrain, Ytrain, # выборка для обучения
    cat_features=np.where(X.dtypes != np.float)[0], # категориальные фичи
    eval_set=(Xtest, Ytest), # выорка для проверки на тесте
    verbose=False, # не выводить логи
    plot=True, # выводить графики
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))